In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
file_path = '/content/drive/My Drive/output.csv'
df = pd.read_csv(file_path)
df.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import tensorflow as tf

In [ ]:
#extracting features
features = ['avg_transaction_value', 'token_transfers_count', 'gas_used', 'transaction_count']
X = df[features].values
y = df['avg_gas_price'].values

In [ ]:
#normalizing
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(y.reshape(-1, 1))

# splitting the data into test, train and eval
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)

# reshaping input
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))

In [ ]:
#custom loss function that penalizes sub-zero predictions
def custom_mse_loss(y_true, y_pred):
    squared_difference = tf.square(y_true - y_pred)
    penalty = tf.square(tf.nn.relu(-y_pred))
    total_loss = tf.reduce_mean(squared_difference + penalty)

    return total_loss


# building lstm model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss=custom_mse_loss)

# training model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
# evaluation metrics
loss = model.evaluate(X_test, y_test)
print('Custom Loss:', loss)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
print('Mean Absolute Error:', mae)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# predicted gas prices
y_pred = model.predict(X_test)

y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1)).ravel()
y_pred_actual = scaler.inverse_transform(y_pred).ravel()

# plotting actual vs. predicted
plt.figure(figsize=(10, 6))
plt.plot(y_test_actual, label='Actual')
plt.plot(y_pred_actual, label='Predicted')
plt.xlabel('Time')
plt.ylabel('Gas Price')
plt.title('Actual vs. Predicted Gas Prices')
plt.legend()
plt.show()

# residuals plot
residuals = y_test_actual - y_pred_actual
plt.figure(figsize=(10, 6))
plt.plot(residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Time')
plt.ylabel('Residuals')
plt.title('Residuals Plot')
plt.show()

# density plot of residuals
plt.figure(figsize=(10, 6))
sns.histplot(residuals, kde=True)
plt.xlabel('Residuals')
plt.ylabel('Density')
plt.title('Density Plot of Residuals')
plt.show()
